In [5]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import URLError
from copy import deepcopy
import numpy as np
import pickle
import time
import undetected_chromedriver as uc

In [69]:
# Collecting all market_report_links
driver = uc.Chrome()
driver.get("https://www.alliedmarketresearch.com/reports-store")

In [2]:
market_report_links = []
with open("market_report_links.pkl", "rb") as f:
    market_report_links = pickle.load(f)

In [ ]:
count = 1
while True:
    print(count)
    market_report_elements = driver.find_elements(By.CLASS_NAME, "reportpg-sec")
    for market_report_element in market_report_elements:
        media_element = market_report_element.find_element(By.CLASS_NAME, "media-body")
        market_report_links.append(media_element.find_element(By.XPATH, ".//h3/a").get_attribute("href"))
    pagination_element = driver.find_element(By.CLASS_NAME, "pagination")
    page_elements = pagination_element.find_elements(By.CLASS_NAME, "page-item")
    if page_elements[-1].text != "Next":
        break
    current_url = driver.current_url
    page_elements[-1].click()
    WebDriverWait(driver, 10000).until(EC.url_changes(current_url))
    count += 1

In [74]:
with open("market_report_links.pkl", "wb") as f:
    pickle.dump(market_report_links, f)

In [ ]:
# Collecting all soups fro market_report_links

In [3]:
from bs4 import BeautifulSoup
import requests


In [4]:
def get_cagr_from_str(line):
    return float(line.split("%")[0].split()[-1])

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

i = 31087

cached_soups = set(market_report_links[:i])


In [ ]:
while i < len(market_report_links):
    start = time.time()
    url = market_report_links[i]
    if url in cached_soups:
        print(i, "repeated url")
        i += 1
        continue
    response = None
    while not response or response.status_code != 200:
        response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    with open("cached_stocks_soups/{}.pkl".format(url.split('/')[-1]), "wb") as f:
        pickle.dump(soup, f)
    end = time.time()
    print(i, end - start)
    i += 1

In [18]:
# ignore_words = set()
# ignore_words.add("inc")
# ignore_words.add("inc.")
# ignore_words.add("corporation")
# ignore_words.add("se")
# ignore_words.add("ag")
# ignore_words.add("ltd")
# ignore_words.add("llc")
# ignore_words.add("se.")
# ignore_words.add("ag.")
# ignore_words.add("ltd.")
# ignore_words.add("llc.")
# ignore_words.add("systems")
# ignore_words.add("gmbh")
# ignore_words.add("gmbh.")
# ignore_words.add("technologies")
# ignore_words.add("co")
# ignore_words.add("co.")
# ignore_words.add("company")
# ignore_words.add("ag")
# ignore_words.add("n.v")
# ignore_words.add("ag.")
# ignore_words.add("n.v.")
# ignore_words.add("incorporated")
# ignore_words.add("limited")
# ignore_words.add("plc")
# ignore_words.add("lp")
# ignore_words.add("kgaa")
# ignore_words.add("plc.")
# ignore_words.add("lp.")
# ignore_words.add("kgaa.")
# ignore_words.add("group")
# ignore_words.add("a/s")
# ignore_words.add("emc")
# ignore_words.add("emc.")
# ignore_words.add("&")
# ignore_words.add("and")
# ignore_words.add("holdings")
# ignore_words.add("ab")
# ignore_words.add("kg")
# ignore_words.add("corp")
# ignore_words.add("s.p.a")
# ignore_words.add("s.a")
# ignore_words.add("ab.")
# ignore_words.add("kg.")
# ignore_words.add("corp.")
# ignore_words.add("s.p.a.")
# ignore_words.add("s.a.")
# ignore_words.add("industries")
# ignore_words.add("pvt")
# ignore_words.add("pvt.")
# ignore_words.add("private")

# def sanitize(company):
#     company_str_split = company.lower().strip().strip(".").split()
#     new_str = []
#     for word in company_str_split:
#         if word not in ignore_words:
#             new_str.append(word)
#     return_str = " ".join(new_str)
#     return return_str

In [1]:
from os import listdir
import numpy as np

In [2]:
files = [f for f in listdir("cached_stocks_soups")]

In [160]:
cagr_present_cases = []
special_cases = []

In [ ]:
to_sift = files
start = time.time()
for i in range(len(to_sift)):
    if i % 100 == 0:
        print(i, time.time() - start)
        start = time.time()
    if to_sift[i][-4:] != ".pkl":
        continue
    soup = None
    with open("cached_stocks_soups/" + to_sift[i], "rb") as f:
        soup = pickle.load(f)
    text_body_element = soup.find(id="roOne")
    if not text_body_element:
        continue
    lines = text_body_element.findChildren("p")
    found = False
    for j in range(min(5, len(lines))):
        if "cagr" in lines[j].text.lower() and "%" in lines[j].text and not "cagr %" in lines[j].text.lower():
            found = True
            break
    if not found:
        lines = text_body_element.findChildren("h2")
        if len(lines) != 0 and "cagr" in lines[0].text.lower() and "%" in lines[0].text and not "cagr %" in lines[0].text.lower():
            found = True
    if found:
        cagr_present_cases.append(to_sift[i])
    else:
        special_cases.append(to_sift[i])

In [163]:
len(cagr_present_cases)

4717

In [ ]:
cagr_present_cases

In [151]:
soup = None
with open("cached_stocks_soups/" + 'europe-gene-editing-market-A22058.pkl', "rb") as f:
    soup = pickle.load(f)
text_body_element = soup.find(id="roOne")
lines = text_body_element.findChildren("p")

In [152]:
found = False
for j in range(len(lines)):
    if "cagr" in lines[j].text.lower() and "%" in lines[j].text:
        found = True
        break

In [153]:
lines[j].text

'COVID-19 impact and revenue opportunity to provide benefits to industry stakeholders with the critical insights to assess the Europe gene editing market.Report Description: The report will cover market trends, size & revenue growth, segmentation, country analysis, competitive landscape, company market shares, and development strategies for this market. The market overview section of the report will cover all aspects of the market in terms of qualitative insights. The market size section will cover country-wide market size and forecast, which will include the impact of the COVID-19. The study further outlines the market segments, which are categorized into submarkets. The country-wise breakdowns are provided along with the size of the market. Competitive landscape includes the detailed assessment on companies in terms of product/service offerings, financial performance, overall business performance, and development strategies. The Europe gene editing market section of the report will b

In [154]:
j

0

In [ ]:
one = set()
one.add("lamea")
one.add("italy")
one.add("france")
one.add("china")
one.add("brazil")
one.add("europe")
one.add("japan")
one.add("spain")
one.add("india")
one.add("uk")
one.add("germany")
one.add("mexico")
one.add("australia")
one.add("canada")
one.add("africa")
one.add("uae")
one.add("belgium")
one.add("switzerland")
one.add("netherlands")
one.add("thailand")
one.add("russia")
one.add("poland")
one.add("argentina")
one.add("singapore")
one.add("vietnam")
one.add("malaysia")
one.add("indonesia")
one.add("chile")
one.add("egypt")
one.add("eygpt")
one.add("algeria")
one.add("philippines")
one.add("turkey")
one.add("kenya")
one.add("sweden")
one.add("ireland")
one.add("venezuela")
one.add("nigeria")
one.add("norway")
one.add("taiwan")
one.add("hungary")
one.add("finland")
one.add("denmark")
one.add("tunisia")
one.add("ukraine")
one.add("qatar")
one.add("us")
one.add("slovakia")
one.add("itlay")
one.add("austria")

two = set()
two.add(("asia", "pacific"))
two.add(("south", "korea"))
two.add(("middle", "east"))
two.add(("u", "k"))
two.add(("u", "s"))
two.add(("north", "america"))
two.add(("saudi", "arabia"))
two.add(("south", "africa"))
two.add(("latin", "america"))
two.add(("united", "kingdom"))
two.add(("the", "netherlands"))
two.add(("new", "zealand"))
two.add(("south", "arabia"))

three = set()
three.add(("united", "arab", "emirates"))
three.add(("south", "east", "asia"))

for file_name in special_cases:
    words = file_name.split('-')
    if words[0] in one or tuple(words[:2]) in two or tuple(words[:3]) in three:
        continue
    print(file_name)

In [162]:
from telegram import Bot

TELEGRAM_TOKEN = '6676215052:AAGgO_Z2WPubhsOAN2_KU7GAeXD9H82ZWw4'
CHAT_ID = '6625944042'

bot = Bot(token=TELEGRAM_TOKEN)

await bot.send_message(chat_id=CHAT_ID, text="done")

Message(channel_chat_created=False, chat=Chat(first_name='Shrikar', id=6625944042, type=<ChatType.PRIVATE>), date=datetime.datetime(2023, 10, 15, 23, 34, 58, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='Fragrantica-complete-pinger', id=6676215052, is_bot=True, username='Fragrantica_complete_bot'), group_chat_created=False, message_id=30, supergroup_chat_created=False, text='done')